In [13]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_feather('./data/df.feather')
df.replace([np.inf, -np.inf], np.nan, inplace=True)  # 处理 inf 值
df.fillna(0, inplace=True)
df.head()

In [ ]:
df.columns.tolist()

In [ ]:
X_train, y_train = [], []
X_test, y_test = [], []

features =['count', 'std_7', 'prc', 'vol', 'cap', 'ret',]
step = 7
split_date = '2021-07-01'

for ticker_f, data in tqdm(df.groupby('ticker_f')):
    data.sort_values('date', inplace=True)
    
    for i in range(step, len(data), 1):
        _X = data.iloc[i-step:i, data.columns.isin(features)].values
        _y = int(data.iloc[i, data.columns.isin(['30d_return'])].item() >= 0)
        
        if data.iloc[i, data.columns.get_loc('date')] < pd.to_datetime(split_date):
            X_train.append(_X)
            y_train.append(_y)
        else:
            X_test.append(_X)
            y_test.append(_y)
        
    # break

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [12]:
data = {
    "X_train": X_train,
    "X_test": X_test,
    "y_train": y_train,
    "y_test": y_test
}

import pickle
with open('./data/data.pkl', 'wb') as f:
    pickle.dump(data, f)
